# Auto Encoder

# Importing the libraries

In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the Dataset

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None,
                     engine='python', encoding='latin')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users =  pd.read_csv('ml-1m/users.dat', sep='::', header=None,
                     engine='python',encoding='latin')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings =  pd.read_csv('ml-1m/ratings.dat', sep='::', header=None,
                       engine='python', encoding='latin')
ratings.tail()

,0,1,2,3
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


# Preparing the training set and test set

In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)
training_set = np.array(training_set, dtype='int')
print(training_set)

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [6]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)
test_set = np.array(test_set, dtype='int')

# Getting the no of users and movies

In [7]:
nb_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
nb_users

943

In [8]:
nb_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))
nb_movies

1682

# Converting the data into an array with users in lines and movies in columns


In [9]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:,  0] == id_users]
        id_ratings = data[:, 2][data[:,  0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [10]:
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors


In [11]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the architecture of nueral network


In [12]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [13]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)


# Training the SAE

In [2]:
nb_epochs = 200
for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        Input = Variable(training_set[id_user]).unsqueeze(0)
        target = Input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(Input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0] * mean_corrector)
            s += 1.
            optimizer.step()
    print('Epoch: ' + str(epoch) + ' Loss: ' + str(train_loss / s))


NameError: name 'nb_users' is not defined

# Testing the SAE

In [1]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    Input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(Input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0] * mean_corrector)
        s += 1.
print('Test Loss: ' + str(test_loss / s))


NameError: name 'nb_users' is not defined